In [11]:
import pandas as pd
import csv
import re

df2 = pd.read_csv(r"OneDrive - Nanyang Technological University/TFIPGA/Thomson Capstone/scotch_review2020.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'OneDrive - Nanyang Technological University/TFIPGA/Thomson Capstone/scotch_review2020.csv'

In [7]:
def drop_non_review_rows(df, review_term="Review #"):
    """Drops rows where the first column does not contain the specified review term (case-insensitive).

    Args:
        df (pd.DataFrame): The DataFrame to process.
        review_term (str, optional): The term to search for (defaults to "review").

    Returns:
        pd.DataFrame: The DataFrame with non-review rows removed.
    """

    def has_review(row):
        # Check if the first column (Title) contains the review term (case-insensitive)
        return review_term.lower() in str(row['Title']).lower()

    return df.loc[df.apply(has_review, axis=1)]


# Assuming you have your DataFrame named 'df'
filtered_df = drop_non_review_rows(df)

filtered_df = filtered_df.drop(["URL","Content"],axis=1)
#filtered_df = filtered_df.iloc[:, [0, -1]]  # Select desired columns by position

# Save the filtered DataFrame to a new CSV file (optional)
filtered_df.to_csv('filtered_reviews.csv', index=False)

print(filtered_df)
print("Data cleaning completed! Filtered data saved to filtered_reviews.csv")



NameError: name 'df' is not defined

In [8]:
#df_drop_columns = pd.read_csv(r"C:\Users\N2397\OneDrive - Nanyang Technological University\TFIPGA\Thomson Capstone\filtered_reviews_with_TC2.csv")

# Assuming your DataFrame is named `df_drop_columns` and the column containing review text is named `Title`

# Extract the desired part of the title using regular expressions

pattern = r"Review #\d+(?:[:,\.]*)(.*)"  # Matches "Review #xxx:" followed by any character sequence


filtered_df['Title'] = filtered_df['Title'].str.extract(pattern, expand=True)

# Print the DataFrame with the modified title
print(filtered_df)

# Save the filtered DataFrame to a new CSV file (optional)
filtered_df.to_csv('filtered_reviews_drop_review.csv', index=False)

NameError: name 'filtered_df' is not defined

In [ ]:
def remove_asterisks(text):
  """
  Removes asterisks (*) from the input text.

  Args:
      text (str): The text to remove asterisks from.

  Returns:
      str: The text with asterisks removed.
  """
  if pd.isna(text):  # Handle missing values in the "Comments" column
    return text
  else:
    return re.sub(r"\*", "", text)

# Create a new column named "Cleaned Comments" with asterisks removed
filtered_df["Cleaned Comments"] = filtered_df["Comments"].apply(remove_asterisks)

# Print the DataFrame with the new column
print(filtered_df)



In [ ]:
#filtered_df.to_csv('filtered_no_asterisks.csv', index=False)
temp_df = filtered_df.copy()

In [ ]:
def extract_tasting_notes(text):
  """
  Extracts tasting notes (Nose, Taste, Finish) and score from the input text.

  Args:
      text (str): The text to extract tasting notes and score from.

  Returns:
      dict: A dictionary containing the extracted tasting notes (Nose, Taste, Finish)
            and score (as the number before "/10"), or an empty dictionary if no notes are found.
  """
  notes = {}
  score_match = re.search(r"\d+/\d+$", text)  # Regex to find "x/10" pattern
  if score_match:
    score = score_match.group().split("/")[0]  # Extract only the number before "/10"
    notes["Score"] = score

  for part in text.split("\n"):
    # Split by colon (":") to get the part name (e.g., Nose) and description
    if ":" in part:
      name, description = part.split(":", 1)
      notes[name.strip()] = description.strip()
  return notes

# Apply the function, check for empty notes, and create new columns
def assign_tasting_notes(row):
  notes = extract_tasting_notes(row["Cleaned Comments"])
  if notes:  # Check if notes dictionary is not empty
    return pd.Series({"Title": row.get("Title", ""), "Nose": notes.get("Nose", ""), "Taste": notes.get("Taste", ""), "Finish": notes.get("Finish", ""), "Score": notes.get("Score", "")})
  else:
    return pd.Series({"Title": "", "Nose": "", "Taste": "", "Finish": "", "Score": ""})  # Return empty series if no notes
filtered_df = filtered_df.apply(assign_tasting_notes, axis=1)

def format_score(score):
  """
  Removes "/10" from the score string (if present).

  Args:
      score (str): The score string (e.g., "9").

  Returns:
      str: The score without "/10", or an empty string if no score is found.
  """

# Save the DataFrame to a CSV file
filtered_df.to_csv("extracted_tasting_notes.csv", index=False)

# Print confirmation message (optional)
print("Tasting notes and score saved to extracted_tasting_notes.csv")


In [ ]:
def new_extract_tasting_notes(text):
  """
  Extracts tasting notes (Nose, Taste, Finish) and score from the input text.

  Args:
      text (str): The text to extract tasting notes and score from.

  Returns:
      dict: A dictionary containing the extracted tasting notes (Nose, Taste, Finish)
            and score (as the number before "/10"), or an empty dictionary if no notes are found.
  """
  notes = {}
  score_match = re.search(r"\d+/\d+$", text)  # Regex to find "x/10" pattern
  print(score_match)
  if score_match:
    score = score_match.group().split("/")[0]  # Extract only the number before "/10"
    notes["Score"] = score

  for part in text.split("\n"):
    # Split by colon (":") to get the part name (e.g., Nose) and description
    if ":" in part:
      name, description = part.split(":", 1)
      notes[name.strip()] = description.strip()
  return notes

new_extract_tasting_notes(temp_df["Cleaned Comments"][425])

In [ ]:
# Function to check if a value is a number
def is_number(value):
    return isinstance(value, (int, float))

# Function to extract the first number from a string
def extract_first_number(text):
    numbers = re.findall(r'\d+', str(text))
    return int(numbers[0]) if numbers else None

# Apply the transformation
filtered_df['Score'] = filtered_df['Score'].apply(lambda x: extract_first_number(x) if not is_number(x) else x)
filtered_df.to_csv('clean_score.csv')

In [ ]:
display(filtered_df)

In [ ]:
# Find the rows with null values
null_rows = filtered_df[filtered_df.isnull().any(axis=1)]

num_null_rows = null_rows.shape[0]

# Print the number of null rows
print(f"Number of null rows: {num_null_rows}")

# Print the null rows
print(null_rows)